In [10]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms,models

In [11]:
# Set device to CUDA if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [12]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [13]:
# train_data = DataLoader(TensorDataset(train_data), batch_size=18, shuffle=True)
# test_data = DataLoader(TensorDataset(test_data), batch_size=18, shuffle=False)

In [14]:
data_dir = 'data/antsbees'
image_datasets ={}
image_datasets['train'] = datasets.ImageFolder(os.path.join(data_dir,'train'), transform=transform)
image_datasets['val']=datasets.ImageFolder(os.path.join
                                             (data_dir,'val'),
                                             transform=transform)
train_loader=DataLoader(image_datasets['train'],batch_size=128,shuffle=True)
test_loader=DataLoader(image_datasets['val'],batch_size=128,shuffle=False)


In [15]:
#load pre train model
model=models.resnet18(pretrained=True).to(device)  #weights='IMAGENET1K_V1'
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:

#freeze all the layers initially
for param in model.parameters():
    param.requires_grad=False

In [17]:
#replace final fully connected layer
num_features=model.fc.in_features
model.fc=nn.Linear(num_features,2).to(device)
criterion=nn.CrossEntropyLoss()

In [18]:
optimizer=optim.Adam(model.fc.parameters(),lr=0.001)

for epoch in range(20):
    model.train()
    for image,label in train_loader:
        image,label=image.to(device),label.to(device)
        output=model(image)
        loss=criterion(output,label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
correct=0.0
total=0.0
model.eval()
with torch.no_grad():
    for image,label in test_loader:
        image,label=image.to(device),label.to(device)
        output=model(image)
        max,predicted=torch.max(output,1)
        correct+= (predicted==label).sum().item()
        total+=label.size(0)
    print(f"test accuracy:{(correct/total)*100}")

test accuracy:94.77124183006535


In [19]:
# Verify GPU usage
if device.type == 'cuda':
    print("GPU memory usage:")
    print(torch.cuda.memory_summary(device=device, abbreviated=True))
else:
    print("Running on CPU.")

GPU memory usage:
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  78527 KiB |    920 MiB | 145205 MiB | 145129 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  78527 KiB |    920 MiB | 145205 MiB | 145129 MiB |
|---------------------------------------------------------------------------|
| Requested memory      |  76076 KiB |    917 MiB | 145174 MiB | 145100 MiB |
|---------------------------------------------